In [ ]:
function plot_kdist_graph(p_snnmat::SparseMatrixCSC{Float64, Int64}, k)
    kdist_arr = Float64[];

    for i in collect(1:size(p_snnmat,2))
        #Q = DataStructures.PriorityQueue(Int64, Float64, Base.Order.Reverse); # Base.Order.Reverse -> highest to lowest
        Q = DataStructures.PriorityQueue{Int64, Float64}(Base.Order.Reverse);
        nnzindices_i = p_snnmat[:,i].nzind;
        for j in nnzindices_i
            Q[j] = p_snnmat[j,i];#append similarity
        end
        c = 1;
        xx = 0;
        while c < k && length(Q) > 0 # k-th extracted item will be the k-th lowest similarity (~ k-th highest distance)
            xx = DataStructures.peek(Q)[2];
            DataStructures.dequeue!(Q);
            c += 1;
        end

        push!(kdist_arr, 1.0 - xx);
    end
    sort!(kdist_arr, rev=true); # desc order of distances 
    
    return kdist_arr
end

## DIstributed Shared Nearest Neighbor experiments

In [ ]:
if length(workers()) > 1
    println("Removing previous workers...")
    rmprocs(workers())
end

addprocs(["158.251.93.5:3308","158.251.93.5:3307","158.251.93.5:3306","158.251.93.5:3305",])

#addprocs(["158.251.93.5:3308","158.251.93.5:3307","158.251.93.5:3306","158.251.93.5:3305",
#        "158.251.93.5:3304","158.251.93.5:3303","158.251.93.5:3302","158.251.93.5:3301",])

@everywhere include("/workspace/distributed_clustering/julia/src/dsnn_IO.jl")
@everywhere include("/workspace/distributed_clustering/julia/src/dsnn_KNN.jl")
@everywhere include("/workspace/distributed_clustering/julia/src/dsnn_SNN.jl")
@everywhere include("/workspace/distributed_clustering/julia/src/dsnn_Master.jl")

using PyCall
@pyimport clustering_scores as cs #clustering_scores.py must be in the path.
using JLD

In [ ]:
results = Dict{String,Any}();
DATA_PATH = "/workspace/cure_large.dat";
partitions = DSNN_Master.generate_partitions(length(workers()), 8100); # N must be extracted from the data.
overall_parameters = Dict{String, Any}("k"=>100, "snn_eps"=>0.988, "snn_minpts"=>5, "k_appindex"=>200);
DSNN_Master.start(results, DATA_PATH, partitions, 0.1, 5, worker_params=overall_parameters);

The _start_ method sets the value of the dictionary _results_ by adding the following values:

* stage1_corepoints: Union of all corepoints identified by the Workers.
* stage1_sampled: Union of all the data points sampled by the Workers.
* stage1_graph: SNN Graph built with __all__ the data returned by the Workers.
* stage2_labels: Labels generated by the Workers after _stage 2_.

In [ ]:
using Plots
pyplot()
D = DSNN_IO.sparseMatFromFile(DATA_PATH, l2normalize=false);

colors = fill(1, size(D,2));
colors[results["stage1_corepoints"]] = 2;
plt = scatter(D[1,:], D[2,:], marker=(0.05,:o,4), legend=false, fmt = :png)
scatter!(plt, D[1,results["stage1_corepoints"]], D[2,results["stage1_corepoints"]], marker=(0.8,:o,4), legend=false, fmt = :png)

In [ ]:
Stats.countmap(results["stage2_labels"])

In [ ]:
using Plots
pyplot()

worker_assignment = find(x -> x==11, partitions);
Dw = DSNN_IO.sparseMatFromFile(DATA_PATH, assigned_instances=worker_assignment, l2normalize=true);

@time begin
println("Computing snn similarity...")
k=100;
#k_ap = 100; epsilon = 0.01;
#apix = DSNN_KNN.initialAppGraph(Dw, k_ap, epsilon, k_ap*2);
#DSNN_KNN.improve_graph!(apix, Dw, k_ap, epsilon, k_ap*2);

#knnmat_ap, nbrhd_len = DSNN_KNN.get_knnmatrix(apix, k)#, sim_threshold = 0.15);
#snnmat_ap = DSNN_KNN.get_snnsimilarity(knnmat_ap, nbrhd_len)
knnmat_ap = DSNN_KNN.get_knn(Dw, 30);
snnmat_ap = transpose(knnmat_ap) * knnmat_ap;
sim_graph = DSNN_KNN.get_snngraph(knnmat_ap, snnmat_ap);

#knnmat_ap, nbrhd_len = DSNN_KNN.get_knnmatrix(apix, k, binarize=false);#, sim_threshold = 0.15);
#sim_graph = DSNN_KNN.get_knngraph(knnmat_ap);

println("SIM graph with nnz%",round(length(sim_graph.nzval)/size(sim_graph,1)^2,4))
end

@time begin
#eps=0.9500;minpts=5;
eps=0.7;minpts=5;
println("Executing snn clustering with eps:",eps," and minpts:", minpts)
results = DSNN_SNN.snn_clustering(eps, minpts, sim_graph);

labels_found = fill(0, size(results["labels"],1));
for c in collect(1:size(results["labels"],2))
    for i in results["labels"][:,c].nzind
        labels_ap[i] = results["clusters"][c]; # extract the right assigned label name
    end
end
end
# reporting results
println("Nr. of groups (+noise):",length(unique(labels_found)))
println(Stats.countmap(labels_found))
println("Amount of noise:",round(length(results["labels"][:,1].nzind)/size(results["labels"],1),2))
Dw_unnorm = DSNN_IO.sparseMatFromFile(DATA_PATH, assigned_instances=worker_assignment, l2normalize=false);
Plots.scatter(Dw_unnorm[1,:], Dw_unnorm[2,:], marker=(0.2,:o,4), groups=labels_found, legend=false, fmt = :png)
Plots.scatter!(Dw_unnorm[1,results["corepoints"]], Dw_unnorm[2,results["corepoints"]], marker=(0.9,:<,7), legend=false, fmt = :png)

In [ ]:
kdist_graph = plot_kdist_graph(snn_graph, k);
Plots.histogram(kdist_graph)

In [ ]:
?DSNN_KNN.get_snngraph

In [ ]:
?DSNN_KNN.get_knngraph

In [ ]:
dot(Dw[:,1], Dw[:,2])

In [ ]:
Dw[:,1]

In [ ]:
Dw[:,2]

In [ ]:
sqrt((Dw[1,1]-Dw[1,2])^2+(Dw[2,1]-Dw[2,2])^2)

In [ ]:
sqrt((Dw[1,1]-Dw[1,101])^2+(Dw[2,1]-Dw[2,101])^2)

In [ ]:
results["labels"][:,1]